## ChatMessageHistory

In [2]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi"));
await history.addMessage(new AIMessage("What can I do for you?"));

const messages = await history.getMessages();

console.log(messages);


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


## 手动维护 chat history

In [3]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [4]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const chatModel = new ChatOpenAI();
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [5]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));

const res1 = await chain.invoke({
    history_message: await history.getMessages()
})


In [6]:
res1

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello Kai! It's nice to meet you. How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello Kai! It's nice to meet you. How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 17, promptTokens: 71, totalTokens: 88 },
    finish_reason: "stop"
  }
}

In [7]:
await history.addMessage(res1)
await history.addMessage(new HumanMessage("What is my name?"));

In [8]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello Kai! It's nice to meet you. How can I assist you today?",
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello Kai! It's nice to meet you. How can I assist you today?",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 17, promptTokens: 71, totalTokens: 88 },
      finish_reason: "stop"
    }
  },
  HumanMessage {
    lc_serializable: true,
    l

In [9]:
const res2 = await chain.invoke({
    history_message: await history.getMessages()
})

In [10]:
res2

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Kai.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Kai.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 101, totalTokens: 106 },
    finish_reason: "stop"
  }
}

In [11]:
console.log(res2.content)

Your name is Kai.


## 自动维护 chat history

In [12]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const chatModel = new ChatOpenAI();
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});

In [13]:
const res1 = await chainWithHistory.invoke({
    input: "hi, my name is Kai"
},{
    configurable: { sessionId: "none" }
})

In [15]:
res1

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello, Kai! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello, Kai! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 11, promptTokens: 33, totalTokens: 44 },
    finish_reason: "stop"
  }
}

In [16]:
const res2 = await chainWithHistory.invoke({
    input: "我的名字叫什么？"
},{
    configurable: { sessionId: "none" }
})

In [17]:
res2

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "您的名字是Kai。有什么我可以帮助您的吗？",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "您的名字是Kai。有什么我可以帮助您的吗？",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 23, promptTokens: 61, totalTokens: 84 },
    finish_reason: "stop"
  }
}

In [18]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello, Kai! How can I assist you today?",
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello, Kai! How can I assist you today?",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 11, promptTokens: 33, totalTokens: 44 },
      finish_reason: "stop"
    }
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "我的名字叫什么？",
     

## 自动生成 chat history 摘要

In [2]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

In [3]:
const summaryModel = new ChatOpenAI();
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])


In [7]:
const res = await summaryChain.invoke({
    summary: "",
    new_lines: "I'm 18"
})

In [8]:
console.log(res)

So far, we've established that the conversation involves someone who is 18 years old.


In [9]:
await summaryChain.invoke({
    summary: res,
    new_lines: "I'm male"
})

"The conversation now involves an 18-year-old male."

In [17]:
const chatModel = new ChatOpenAI();
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
    ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            console.log(summary, input, messages, new_lines, newSummary)
            history.clear()
            summary = newSummary      
        }
    })
])


In [11]:
await chatChain.invoke("我现在饿了")

 您想吃什么？我能帮您推荐一些食物。 [
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我现在饿了", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我现在饿了",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "您想吃什么？我能帮您推荐一些食物。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "您想吃什么？我能帮您推荐一些食物。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
] Human: 我现在饿了
AI: 您想吃什么？我能帮您推荐一些食物。 The human is now stating that they are hungry. The AI is offering to help recommend some food for the human to eat.


"您想吃什么？我能帮您推荐一些食物。"

In [12]:
console.log(summary)

The human is now stating that they are hungry. The AI is offering to help recommend some food for the human to eat.


In [13]:
await chatChain.invoke("我今天想吃方便面")

The human is now stating that they are hungry. The AI is offering to help recommend some food for the human to eat. 方便面是一个很不错的选择！想要尝试不同口味的方便面吗？我可以帮你推荐一些口味。 [
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "我今天想吃方便面",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我今天想吃方便面",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "方便面是一个很不错的选择！想要尝试不同口味的方便面吗？我可以帮你推荐一些口味。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "方便面是一个很不错的选择！想要尝试不同口味的方便面吗？我可以帮你推荐一些口味。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
] Human: 我今天想吃方便面
AI: 方便面是一个很不错的选择！想要尝试不同口味的方便面吗？我可以帮你推荐一些口味。 The human expresses a desire to eat instant noodles today. The AI agrees that instant noodles are a good c

"方便面是一个很不错的选择！想要尝试不同口味的方便面吗？我可以帮你推荐一些口味。"

In [14]:
console.log(summary)

The human expresses a desire to eat instant noodles today. The AI agrees that instant noodles are a good choice and offers to recommend different flavors for the human to try.


In [15]:
import { RunnableMap } from "@langchain/core/runnables"

const mapChain = RunnableMap.from({
    a: () => "a",
    b: () => "b"
})

const res = await mapChain.invoke()

In [16]:
res

{ a: "a", b: "b" }